In [1]:
import requests
import json
import openai

openai.api_key = open(".tokens/openai", "r").read()
print("Started")

Started


In [3]:
# Uses anki-connect to connect to anki
response = requests.post('http://localhost:8765', data='{"action": "deckNames", "version": 6}', headers={'Content-type': 'application/json'})
print(response.json())

{'result': ['Driving License', 'English', 'German', 'German::A2_Wortliste_Goethe', 'German::B1_Wortliste_DTZ_Goethe', 'German::Deutsch B1', 'German::Deutsch B1+', 'German::DW Nicos Weg B1', 'Standard'], 'error': None}


In [4]:
def invoke(method, **params):
    request_data = json.dumps({
        'action': method,
        'params': params,
        'version': 6
    })

    response = requests.post('http://localhost:8765', data=request_data)
    response_data = json.loads(response.text)
    print(response_data)

    if response_data.get('error'):
        raise Exception(f"Error: {response_data['error']}")
    else:
        return response_data['result']

In [19]:
task = """
You create cards for Spaced Repetition to learn german as a foreign language. There is an existing structure for cards, for example:

```json
[
    {
        "Wort_DE": "melden",
        "Wort_EN": "to report, register, put one's hand up, volunteer",
        "Verbformen": "meldet, meldete, hat gemeldet",
        "Satz1_DE": "Ich habe schon zweimal angerufen. - Aber es meldet sich niemand.",
        "Satz1_EN": "I have already called twice. - But nobody answers.",
        "Satz2_DE": "Den Unfall müssen Sie der Versicherung melden.",
        "Satz2_EN":"You must report the accident to the insurance company.",
        "Satz3_DE":"Ich melde mich auf Ihre Anzeige. - Sie wollen ein Sofa verkaufen?",
        "Satz3_EN": "I'm reporting to your ad. - You want to sell a sofa?",
        "Satz4_DE":"Warum hast du dich so lange nicht gemeldet? Warst du krank?",
        "Satz4_EN": "Why have not you been in contact for so long? Were you sick?"
    },
    {
        "Wort_DE": "Pflicht",
        "Wort_EN": "Duty; Obligation; Mandatory",
        "Artikel": "die",
        "Plural": "-en",
        "Satz1_DE": "Eltern haben die Pflicht, ihre Kinder vor Schaden zu bewahren.",
        "Satz1_EN": "Parents have an obligation to protect their children from harm.",
        "Satz2_DE": "Er hatte seine vertraglichen Pflichten erfüllt.",
        "Satz2_EN": "He had met his contractual obligations."
    }
]
```

Please don't output anything else apart from the JSON. The JSON should be valid. The JSON should be an array of objects.

Use the same structure for the new words:
nicken
"""

In [20]:
result = openai.ChatCompletion.create(
  model="gpt-4-1106-preview",
  messages=[
        {"role": "user", "content": task}
    ]
)
content = result.to_dict_recursive()['choices'][0]["message"]["content"]

In [21]:
def clean_content(markdown: str):
    # this method extracts the code from the markdown and returns only the code from the markdown string

    result = ""
    in_code = False
    for line in markdown.split("\n"):
        if line.startswith("```"):
            in_code = not in_code
        elif in_code:
            result += line + "\n"
    return result

In [22]:
# print(content)
cleaned_result = clean_content(content)
# print(cleaned_result)
new_words = json.loads(cleaned_result)
print(new_words)

[{'Wort_DE': 'nicken', 'Wort_EN': 'to nod', 'Verbformen': 'nickt, nickte, hat genickt', 'Satz1_DE': 'Sie nickte zum Gruß.', 'Satz1_EN': 'She nodded in greeting.', 'Satz2_DE': 'Um Zustimmung zu zeigen, nickte er einfach.', 'Satz2_EN': 'To show agreement, he simply nodded.', 'Satz3_DE': 'Jedes Mal, wenn ich eine Frage stellte, nickte er.', 'Satz3_EN': 'Every time I asked a question, he nodded.', 'Satz4_DE': 'Er nickte mir zu, um mir zu signalisieren, dass ich beginnen sollte.', 'Satz4_EN': 'He nodded to me to signal that I should start.'}]


In [88]:
new_words_example = """
[
    {
        "Wort_DE": "infrage kommen",
        "Wort_EN": "to come into question, be considered",
        "Verbformen": "kommt infrage, kam infrage, ist infrage gekommen",
        "Satz1_DE": "Für diese Stelle kommen nur erfahrene Kandidaten infrage.",
        "Satz1_EN": "Only experienced candidates come into question for this position.",
        "Satz2_DE": "Das Auto ist zu teuer, es kommt nicht infrage.",
        "Satz2_EN":"The car is too expensive, it is not being considered.",
        "Satz3_DE":"Alle Möglichkeiten, die infrage kommen, sollten geprüft werden.",
        "Satz3_EN": "All possibilities that come into question should be checked.",
        "Satz4_DE":"Für mich kam es nicht infrage, die Party zu verpassen.",
        "Satz4_EN": "For me, missing the party was out of the question."
    },
    {
        "Wort_DE": "-bar bedeutet Möglich",
        "Wort_EN": "-able means possible",
        "Satz1_DE": "Die Worte 'essbar', 'lesbar', und 'tragbar' zeigen, dass '-bar' 'möglich' bedeutet.",
        "Satz1_EN": "The words 'edible', 'readable', and 'wearable' show that '-able' means 'possible'.",
        "Satz2_DE": "'Waschbar' bedeutet, dass es möglich ist, es zu waschen.",
        "Satz2_EN": "'Washable' means it is possible to wash it."
    },
    {
        "Wort_DE": "Es kam mir darauf an,",
        "Wort_EN": "It mattered to me,",
        "Verbformen": "es kommt mir darauf an, es kam mir darauf an, es ist mir darauf angekommen",
        "Satz1_DE": "Es kam mir darauf an, dass wir alle zusammenbleiben.",
        "Satz1_EN": "It mattered to me that we all stick together.",
        "Satz2_DE": "Es kam mir darauf an, meine Prüfung gut zu bestehen.",
        "Satz2_EN":"It was important to me to pass my exam well.",
        "Satz3_DE":"Es kam mir darauf an, pünktlich zu sein.",
        "Satz3_EN": "It mattered to me to be on time.",
        "Satz4_DE":"Es kam mir darauf an, dass du ehrlich zu mir bist.",
        "Satz4_EN": "It mattered to me that you are honest with me."
    },
    {
        "Wort_DE": "Ich konnte es kaum erwarten, bis...",
        "Wort_EN": "I could hardly wait until...",
        "Verbformen": "kann kaum erwarten, konnte kaum erwarten, hat kaum erwarten können",
        "Satz1_DE": "Ich konnte es kaum erwarten, bis der Film startete.",
        "Satz1_EN": "I could hardly wait for the film to start.",
        "Satz2_DE": "Ich konnte es kaum erwarten, bis wir in den Urlaub fuhren.",
        "Satz2_EN":"I could hardly wait until we went on vacation.",
        "Satz3_DE":"Ich konnte es kaum erwarten, bis das Buch veröffentlicht wurde.",
        "Satz3_EN": "I could hardly wait until the book was published.",
        "Satz4_DE":"Ich konnte es kaum erwarten, bis ich dich wieder sah.",
        "Satz4_EN": "I could hardly wait until I saw you again."
    }
]
"""
new_words = json.loads(new_words)

In [23]:
# Remove None fields
for word in new_words:
    for key in list(word.keys()):
        if word[key] is None:
            del word[key]
    invoke('addNote', 
       note={
           'deckName': 'German::Deutsch B1', 
           'modelName': 'Basic (and reversed card)-75aea', 
           'fields': word, 
           'options': {'allowDuplicate': False}, 
           'tags': ['German', 'B1']})


{'result': 1704638897384, 'error': None}
